# Table of Contents
## 0. load libraries, modules
## 1. Online audiences vs. Offline audiences
## 2. Data store audiences vs. non-data store audiences
## 3. Email-only audiences 
## 4. CID-only audiences 
## 5. Address-only audiences

# 0. load libraries, modules

In [3]:
# import basics
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
pd.set_option('display.max_rows', 500)

# 1. Online audiences vs. Offline audiences

## 1.1 results

total enabled audiences:
21,364

offline audiences: 15,707

online audiences: 5,117


## 1.2 data wrangling

### query all audiences -> enabled, non-FBR

``` mysql
SELECT *
FROM `corp-bi-us-prod.rldb.liveramp_customer_accounts` 
WHERE enabled = true AND customer_link_audience = false;
```

In [ ]:
# load table w/all audiences
lcas = pd.read_csv('/Users/aschul/Documents/Documents - aschul1118mac/Code_Base/Investigations/20200417_AK_analysis/all_LCAs.csv')

### query audience type summary -> enabled, non-FBR

``` myql
SELECT type, count(type) AS type_count
FROM `corp-bi-us-prod.rldb.liveramp_customer_accounts` 
WHERE enabled = true AND customer_link_audience = false
GROUP BY type
ORDER BY type_count DESC
```

In [79]:
aud_sum = pd.read_csv('/Users/aschul/Documents/Documents - aschul1118mac/Code_Base/Investigations/20200417_AK_analysis/all_LCAs_summary.csv')
aud_sum

,type,type_count
0,OfflineLiverampCustomerAccount,15707
1,OnlineLiverampCustomerAccount,5117
2,CidPinMappingLiverampCustomerAccount,206
3,CidAnonymousIdMappingLiverampCustomerAccount,158
4,MuidOnlyLiverampCustomerAccount,75
5,MuidHeidMappingLiverampCustomerAccount,73
6,MatchLiverampCustomerAccount,45
7,OfflineFulfillmentMappingLiverampCustomerAccount,17
8,IdSyncMappingLiverampCustomerAccount,16


## 1.3 data exploration

In [ ]:
lcas.info()

In [ ]:
lcas.head(2)

# 2. Data store audiences vs. non-data store audiences

## 2.1 results
data provider customers - 804 (active customers, defined as having entry in the dms_provider table)

total active audiences - 21,364
___________________

offline audiences - 15,707

offline, data store provider audiences - 546

___________________

online audiences - 5,110

online, data store provider audiences - 1,018
___________________

total data store provider audiences - 1,567



## 2.2 data wrangling

### query all data provider customers -> customer status = enabled

```mysql
SELECT dms_providers.customer_id, customers.id
FROM `corp-bi-us-prod.rldb.dms_providers` AS dms_providers
  INNER JOIN `corp-bi-us-prod.rldb.customers` AS customers
    ON dms_providers.customer_id = customers.id
WHERE customers.status = 1
```

In [68]:
# 804 customers are data providers, defined by having an entry in the dms_provider table and customers.enabled = true

dms_providers = pd.read_csv('/Users/aschul/Documents/Documents - aschul1118mac/Code_Base/Investigations/20200417_AK_analysis/dms_providers.csv')
dms_providers.count()
dms_providers.head(2)

customer_id    804
name           804
dtype: int64

### query all data store audiences -> enabled, non-FBR audiences, customer status = enabled

```mysql
SELECT lr_customer_accounts.id AS audience_id, lr_customer_accounts.display_name AS audience_name, customers.name AS customer_name, customers.id AS customer_id, lr_customer_accounts.type
FROM `corp-bi-us-prod.rldb.dms_segments` AS dms_segments
INNER JOIN `corp-bi-us-prod.rldb.lrc_field_definitions` AS lrc_field_definitions
  ON dms_segments.lrc_field_definition_id = lrc_field_definitions.id
INNER JOIN `corp-bi-us-prod.rldb.liveramp_customer_accounts` AS lr_customer_accounts
  ON lrc_field_definitions.lr_customer_account_id = lr_customer_accounts.id
INNER JOIN `corp-bi-us-prod.rldb.customers` AS customers
  ON lr_customer_accounts.customer_id = customers.id
WHERE lr_customer_accounts.enabled = true AND lr_customer_accounts.customer_link_audience = false AND customers.status = 1
GROUP BY lr_customer_accounts.id, lr_customer_accounts.display_name, customers.name, customers.id, lr_customer_accounts.type
```

In [74]:
dms_audiences = pd.read_csv('/Users/aschul/Documents/Documents - aschul1118mac/Code_Base/Investigations/20200417_AK_analysis/all_DS_non_DS_audiences.csv')
dms_audiences
dms_audiences.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1564 entries, 0 to 1563
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   audience_id    1564 non-null   int64 
 1   audience_name  1564 non-null   object
 2   customer_name  1564 non-null   object
 3   customer_id    1564 non-null   int64 
 4   type           1564 non-null   object
dtypes: int64(2), object(3)
memory usage: 61.2+ KB


# 3. Email-only audiences

## 3.1 Results
### all audiences
enabled, non-FBR audiences where AK = email and customer = enabled

audiences w/AK as raw email - 4,995

audiences w/AK as hashed email - 358

-------------
audiences w/AK as MD5 email - 175

audiences w/AK as SHA256 email - 84

audiences w/AK as SHA1 email - 46

audiences w/AK as unknown hash email - 53

### non-DS audiences
enabled, non-FBR audiences where AK = email and customer = enabled

non DS audiences w/AK set as raw email - 4,918

non-DS audienecs w/AK set as hashed email - 328

-------------
audiences w/AK set as MD5 email - 159

audiences w/AK set as SHA256 email - 78

audiences w/AK set as SHA1 email - 38

audiences w/AK set as unknown hash email - 52

### DS audiences
enabled, non-FBR audiences where AK = email and customer = enabled

DS audiences w/AK set as raw email - 77

DS audiences w/AK set as hashed email - 31

-------------
DS audiences w/AK set as MD5 email - 16

DS audiences w/AK set as SHA256 email - 6

DS audiences w/AK set as SHA1 email - 8

DS audiences w/AK set as unknown hash email - 1

## 3.2 Data wrangling


### 3.2.1 query all audiences with AK like '%email%' -> enabled, non-FBR audiences, enabled customers

```mysql
SELECT *
FROM audience_key_labels
INNER JOIN liveramp_customer_accounts
	ON audience_key_labels.liveramp_customer_account_id = liveramp_customer_accounts.id
INNER JOIN customers
	ON liveramp_customer_accounts.customer_id = customers.id
WHERE audience_key_labels.key_index = 0 AND audience_key_labels.label like '%email%' AND customers.status = 1 AND liveramp_customer_accounts.enabled = true AND liveramp_customer_accounts.customer_link_audience = false
GROUP BY liveramp_customer_accounts.id
```

In [17]:
raw_emails_all_audiences = pd.read_csv('/Users/aschul/Documents/Documents - aschul1118mac/Code_Base/Investigations/20200417_AK_analysis/email_AKs/raw_emails.csv')    

In [130]:
raw_emails_all_audiences.head(5)

,id,liveramp_customer_account_id,key_index,label,id.1,customer_id,data_partner_thrift_identifier,display_name,volume_limit,updated_at,...,evaluation_tos_accepted_by,merged,query_rate_estimate,esp_key,reference_liveramp_customer_account_id,email_alias,custom_billing_rule_id,custom_billing_rule_type,encryption_key,enable_match_reports
0,928,607,0,email,607.0,337461,64,Madison Logic - Email,NaN,2018-10-18 18:30:49,...,NaN,0,NaN,NaN,101819,bombora@liveramp.com,NaN,NaN,e5782f17c88e4f9cc9a6b1b7623d1a0d,1.0
1,4948,1678,0,Md5 Email,1678.0,48144,179,LiveRamp Analysis - Gmail Test Users,NaN,2018-05-02 15:07:36,...,NaN,0,NaN,NaN,NaN,datastoreoperations@liveramp.com,NaN,NaN,97f4b92984b03701b3c54d433a3ffa7d,0.0
2,5038,2098,0,Email,2098.0,522676,221,Arrivalist - One Bird,NaN,2018-05-02 15:07:36,...,NaN,0,NaN,NaN,NaN,arrivalist@liveramp.com,NaN,NaN,1082549ad65243db9746b52448aff0ad,0.0
3,11148,2258,0,Email,2258.0,400648,232,Webbula - CAL,NaN,2018-02-20 18:51:07,...,NaN,0,NaN,NaN,NaN,webbula@liveramp.com,NaN,NaN,b8d11659c18061ec8e72a19fd92c94b3,0.0
4,11358,2288,0,Email,2288.0,400648,235,Webbula - PUB,NaN,2018-02-20 18:51:08,...,NaN,0,NaN,NaN,NaN,webbula@liveramp.com,NaN,NaN,b8d11659c18061ec8e72a19fd92c94b3,0.0


In [18]:
raw_email_values = pd.DataFrame(raw_emails_all_audiences["label"].value_counts())

In [19]:
raw_email_values.info()

<class 'pandas.core.frame.DataFrame'>
Index: 250 entries, Email to MEMBER_EMAIL
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   label   250 non-null    int64
dtypes: int64(1)
memory usage: 3.9+ KB


In [133]:
# normalize email columns (raw, MD5, SHA1, SHA256)
raw_emails_all_audiences["label"].replace({"Email":"email", 
                             "Email1":"email",
                             "Email Address":"email",
                             "EmailAddress":"email", 
                             "Emails":"email",
                             "EMAIL":"email",
                             "Email_1":"email",
                             "EmailAddress":"email",
                             "email_address":"email",
                             "EMAIL_ADDRESS":"email",
                             "Email 1":"email",
                             "Email(s)":"email",
                             "email1":"email",
                             "EMAIL_ID":"email",
                             "email1_MD5":"MD5",
                             "emailmd5":"MD5",
                             "profiles.email_address":"email",
                             "EMAIL_ADDR":"email",
                             "SHA256EMAIL1":"email",
                             "email_addr":"email",
                             "MD5EMAIL":"MD5",
                             "EmailAddress":"email",
                             "EMAIL1":"email",
                             "cust_email_addr":"email",
                             "Email_Address":"email",
                             "md5_email":"MD5",
                             "Email_ID":"email",
                             "EMAIL_ADDR_TXT":"email",
                             "emailAddress":"email",
                             "sha1_email":"SHA1",
                             "MD5Email":"MD5",
                             "email_1":"email",
                             "LRI_G_SHA1EMAIL1":"SHA1",
                             "LRI_G_SHA256EMAIL1":"SHA256",
                             "loweremailmd5":"MD5",
                             "email_sha256":"SHA256",
                             "SHA256_EMAIL":"SHA256",
                             "MD5_Email":"MD5",
                             "LRI_DER_SHA256EMAIL1":"SHA256",
                             "email_SHA-1":"SHA1",
                             "sha256email":"SHA256",
                             "email address":"email",
                             "purch_email_addr":"email",
                             "HashedEmail":"hashed_email",
                             "P_EMAIL_ADDRESS":"email",
                             "Member Email Address":"email",
                             "Email Addresses":"email",
                             "email_md5":"MD5",
                             "emails":"email",
                             "Email Hash":"hashed_email",
                             "customer_email":"email",
                             "EMAIL_ADDRESS_":"email",
                             "SHA 256 Email":"SHA256",
                             "EMAILADDRESS":"email",
                             "LRI_DER_SHA1EMAIL1":"SHA1",
                             "email_hash":"hashed_email",
                             "LRI_G_MD5EMAIL1":"MD5",
                             "EMAIL_SHA256":"SHA256",
                             "PII_EMAIL_ADDRESS":"email",
                             "EMail":"email",
                             "Email address":"email",
                             "SHA1_Email":"SHA1",
                             "LRI_DER_MD5EMAIL1":"MD5",
                             "EMAIL_1":"email",
                             "sha256_email":"SHA256",
                             "SHA256EMAIL":"SHA256",
                             "SHA256Email":"email",
                             "emailaddress":"email",
                             "email_id":"email",
                             "SHA1EMAIL":"SHA1",
                             "hashed email":"hashed_email",
                             "SHA256HASHEDEMAIL":"SHA256",
                             "email2":"email",
                             "EMAIL_ADDRESS_1":"email",
                             "md5email":"MD5",
                             "hashedemail":"hashed_email",
                             "EMAIL_HASHED":"hashed_email",
                             "Broker Email":"email",
                             "HASH_MD5_LOWER_EMAIL":"MD5",
                             "profileEmail":"email",
                             "email_Address":"email",
                             "Hashed_Email":"hashed_email",
                             "PER_EMAIL":"email",
                             "Hashed_Email_Address":"email",
                             "Peloton Order Order Email":"email",
                             "EMAIL,FN,LN,,ZIP,CT,PHONE,":"email",
                             "Hashed_Email_ID_Hex":"hashed_email",
                             "#email_lc_md5":"MD5",
                             "MD5_EMAIL":"MD5",
                             "\EMAIL\"":"email",
                             "Customer Email":"email",
                             "CUSTOMERS_SalesOrder_BILLTOLookup::Email":"email",
                             "profile.email":"email",
                             "HASHED_EMAIL":"hashed_email",
                             "sha256email1":"SHA256",
                             "Hashed Email":"hashed_email",
                             "CUST_EMAIL":"email",
                             "SHA256 Email":"SHA256",
                             "email,fn,ln,st,zip,ct,phone,":"email",
                             "best_email":"email",
                             "LRI_G_EMAIL1":"email",
                             "SHA2_EMAIL":"SHA2",
                             "Md5 Email":"MD5",
                             "md5(email1)":"MD5",
                             "master_email":"email",
                             "HASHED_CUST_EMAIL":"hashed_email",
                             "REAL_EMAIL":"email",
                             "email_sha1":"SHA1",
                             "MD5 EMAIL":"MD5",
                             "Email/Email":"email",
                             "hash_email":"hashed_email",
                             "email md5":"MD5",
                             "Email_address":"email",
                             "Email1_SHA-1":"SHA1",
                             "MD5 Hashed Email":"MD5",
                             "EE_EMAIL_ADDRESS":"email",
                             "Email1 Address":"email",
                             "DMD_EMAIL":"email",
                             "QBOVRT_master_email":"email",
                             "lower(b.email_addr_hash_txt)":"hashed_email",
                             "Email1_SHA256":"SHA256",
                             "HashedEmail1":"hashed_email",
                             "CONTACT_EMAIL_ADDR":"email",
                             "Email ":"email",
                             "EmailAddressmd5":"MD5",
                             "CONTACT EMAIL":"email",
                             "MD5EMAIL1":"MD5",
                             "MD5email":"MD5",
                             "sha256email\",470121,624331,470121,UPHE - Onboarding - Salesforce Data\"":"SHA256",
                             "sha256email\",287259,579949,28236,Swirl McGarryBowen - Onboarding\"":"SHA256",
                             "AX_CUREMAIL_10":"email",
                             "Email ":"email",
                             "SHA256_email":"SHA256",
                             "USA_EMAIL":"email",
                             "Right_email":"email",
                             "MDG EMAIL":"MD5",
                             "email3":"email",
                             "sha256email\",287259,579949,28236,Swirl McGarryBowen - Onboarding\"":"SHA256",
                             "sha256email\",470121,624331,470121,UPHE - Onboarding - Salesforce Data\"":"SHA256",
                             "email_SHA_256":"SHA256",
                             "hashed_emailId":"hashed_email",
                             "EMAIL_ADDRESS1":"email",
                             "email_adr1":"email",
                             "EmailAddressHash_MD5":"MD5",
                             "LRI_G_email_address":"email",
                             "DANDRVINSON@EMAIL.COM":"email",
                             "user_email":"email",
                             "Email_hashed":"hashed_email",
                             "P_EMAIL":"email",
                             "Hashed email":"hashed_email",
                             "SHA250Email":"SHA256",
                             "sha256email\",287259,579949,28236,Swirl McGarryBowen - Onboarding\"":"SHA256",
                             "sha256email\",470121,624331,470121,UPHE - Onboarding - Salesforce Data\"":"SHA256",
                             "Email5":"email",
                             "Orders Customer Email":"email",
                             "Encrypted Email address":"hashed_email",
                             "eMail":"email",
                             "patron_email_addr":"email",
                             "Client_Email":"email",
                             "email_01":"email",
                             "emailhash_md5":"MD5",
                             "Person Account: Email":"email",
                             "Email_Appended":"email",
                             "Email_address_hash":"hashed_email",
                             "sha256email\",470121,624331,470121,UPHE - Onboarding - Salesforce Data\"":"SHA256",
                             "EMAIL_HASH":"hashed_email",
                             "Email ID":"email",
                             "email_billing":"email",
                             "PRIMARY_EMAIL_ADDRESS":"email",
                             "Opportunity Email":"email",
                             "CnPh_1_01_Email":"email",
                             "MD5_email":"MD5",
                             "email_address_256hash":"SHA256",
                             "EMAIL ADDR":"email",
                             "sha256email\",154996,542386,15314,StubHub - Salesforce Onboarding\"":"SHA256",
                             "Hash Email":"hashed_email",
                             "prmry_email_addr":"email",
                             "sha256email\",248499,508256,24412,Caesars - Salesforce RFP\"":"SHA256",
                             "MEMBER_EMAIL":"email",
                             "Sha1_Email":"SHA1",
                             "HASH EMAIL ADDR":"hashed_email",
                             "Sha1_Email":"SHA1",
                             "HASH EMAIL ADDR":"hashed_email",
                             "CUS_EMAIL_ADDRESS":"email",
                             "channels.email.address":"email",
                             "\Email1\"\"":"email",
                             "primaryEmail":"email",
                             "sha256email\",313789,533849,313789,Audi - Salesforce Imports\"":"SHA256",
                             "HASEHD_EMAIL":"hashed_email",
                             "\Email1\"\"":"email",
                             "hash email":"hashed_email",
                             "REGI_EMAIL_ADDR":"email",
                             "Customers Email":"email",
                             "email_addres":"email",
                             "sha256email\",116909,533639,11536,Weight Watchers - Onboarding\"":"SHA256",
                             "EMAIL-sha1":"SHA1",
                             "Email Address 1":"email",
                             "Contact Business Email":"email",
                             "\Email1\"\"":"email",
                             "Personal_email":"email",
                             "sha256email\",154996,542386,15314,StubHub - Salesforce Onboarding\"":"SHA256",
                             "EMAIL_MATCH_ID_1":"email",
                             "MD5 Email":"MD5",
                             "associatedemailaddress":"email",
                             "Email_Hash":"hashed_email",
                             "Email_Hash ":"hashed_email",
                             "EMAIL ADDRESS	":"email",
                             "EMAILADDR":"email",
                             "EMAILADDR ":"email",
                             "Contact: Primary Email":"email",
                             "Email Address - Prospects":"email",
                             "sha_email	":"hashed_email",
                             "sha_email":"hashed_email",
                             "primary_contact_primary_email":"email",
                             "EMAIL_ADDR_1":"email",
                             "Primary_Email_Address":"email",
                             "CUST_EMAIL_ADDR":"email",
                             "AlternateEmail":"email"
                            }, inplace = True)                       

In [21]:
raw_email_values.head(25)

,label
Email,1584
Email1,793
email,758
Email Address,424
Emails,327
EMAIL,230
Email_1,207
EMAIL1,113
EmailAddress,71
email_address,63


In [20]:
raw_emails_all_audiences["label"]

0           email
1       Md5 Email
2           Email
3           Email
4           Email
          ...    
5303        Email
5304       Email1
5305        email
5306        Email
5307     emailmd5
Name: label, Length: 5308, dtype: object

### 3.2.1 query all audiences with AK like '%email%' -> enabled, non-FBR audiences, enabled customers, data store

```mysql
SELECT liveramp_customer_accounts.id AS audience_id, liveramp_customer_accounts.display_name AS audience_name, customers.name AS customer_name, customers.id AS customer_id, liveramp_customer_accounts.type, audience_key_labels.label
FROM dms_segments
INNER JOIN lrc_field_definitions 
	ON dms_segments.lrc_field_definition_id = lrc_field_definitions.id
INNER JOIN liveramp_customer_accounts
 	ON lrc_field_definitions.lr_customer_account_id = liveramp_customer_accounts.id
INNER JOIN customers
	ON liveramp_customer_accounts.customer_id = customers.id
INNER JOIN audience_key_labels
	ON audience_key_labels.liveramp_customer_account_id = liveramp_customer_accounts.id
WHERE liveramp_customer_accounts.enabled = true AND liveramp_customer_accounts.customer_link_audience = false AND customers.status = 1 AND audience_key_labels.key_index = 0 AND audience_key_labels.label like '%email%' 
GROUP BY liveramp_customer_accounts.id
```

In [23]:
raw_emails_DS = pd.read_csv('/Users/aschul/Documents/Documents - aschul1118mac/Code_Base/Investigations/20200417_AK_analysis/email_AKs/raw_emails_DS_audiences.csv')

In [24]:
raw_emails_DS.count()

audience_id      108
audience_name    108
customer_name    108
customer_id      108
type             108
label            108
dtype: int64

In [25]:
raw_emails_DS.head(5)

,audience_id,audience_name,customer_name,customer_id,type,label
0,50116,Bridge - Household (Email),Bridge,506009,OfflineLiverampCustomerAccount,email
1,333819,Illion - Australia,illion,592889,OfflineLiverampCustomerAccount,MD5EMAIL
2,201846,WealthEngine Test,Dstillery,370247,OfflineLiverampCustomerAccount,email
3,51336,TruSignal - DMS - md5,TruSignal,515376,OfflineLiverampCustomerAccount,emailmd5
4,326309,Cinemark - Match Test - testfile,Cinemark,552646,OfflineLiverampCustomerAccount,EMAIL1


In [26]:
raw_emails_DS.count()

audience_id      108
audience_name    108
customer_name    108
customer_id      108
type             108
label            108
dtype: int64

In [27]:
raw_emails_DS_values = pd.DataFrame(raw_emails_DS["label"].value_counts())

In [28]:
raw_emails_DS_values

,label
Email,31
email,18
emailmd5,6
LRI_G_SHA1EMAIL1,6
email_address,4
email_addr,4
LRI_G_MD5EMAIL1,3
Email Address,3
EMAIL,3
"email,fn,ln,st,zip,ct,phone,",2


In [29]:
# normalize email columns (raw, MD5, SHA1, SHA256)
raw_emails_DS["label"].replace({"Email":"email", 
                             "Email1":"email",
                             "Email Address":"email",
                             "EmailAddress":"email", 
                             "Emails":"email",
                             "EMAIL":"email",
                             "Email_1":"email",
                             "EmailAddress":"email",
                             "email_address":"email",
                             "EMAIL_ADDRESS":"email",
                             "Email 1":"email",
                             "Email(s)":"email",
                             "email1":"email",
                             "EMAIL_ID":"email",
                             "email1_MD5":"MD5",
                             "emailmd5":"MD5",
                             "profiles.email_address":"email",
                             "EMAIL_ADDR":"email",
                             "SHA256EMAIL1":"email",
                             "email_addr":"email",
                             "MD5EMAIL":"MD5",
                             "EmailAddress":"email",
                             "EMAIL1":"email",
                             "cust_email_addr":"email",
                             "Email_Address":"email",
                             "md5_email":"MD5",
                             "Email_ID":"email",
                             "EMAIL_ADDR_TXT":"email",
                             "emailAddress":"email",
                             "sha1_email":"SHA1",
                             "MD5Email":"MD5",
                             "email_1":"email",
                             "LRI_G_SHA1EMAIL1":"SHA1",
                             "LRI_G_SHA256EMAIL1":"SHA256",
                             "loweremailmd5":"MD5",
                             "email_sha256":"SHA256",
                             "SHA256_EMAIL":"SHA256",
                             "MD5_Email":"MD5",
                             "LRI_DER_SHA256EMAIL1":"SHA256",
                             "email_SHA-1":"SHA1",
                             "sha256email":"SHA256",
                             "email address":"email",
                             "purch_email_addr":"email",
                             "HashedEmail":"hashed_email",
                             "P_EMAIL_ADDRESS":"email",
                             "Member Email Address":"email",
                             "Email Addresses":"email",
                             "email_md5":"MD5",
                             "emails":"email",
                             "Email Hash":"hashed_email",
                             "customer_email":"email",
                             "EMAIL_ADDRESS_":"email",
                             "SHA 256 Email":"SHA256",
                             "EMAILADDRESS":"email",
                             "LRI_DER_SHA1EMAIL1":"SHA1",
                             "email_hash":"hashed_email",
                             "LRI_G_MD5EMAIL1":"MD5",
                             "EMAIL_SHA256":"SHA256",
                             "PII_EMAIL_ADDRESS":"email",
                             "EMail":"email",
                             "Email address":"email",
                             "SHA1_Email":"SHA1",
                             "LRI_DER_MD5EMAIL1":"MD5",
                             "EMAIL_1":"email",
                             "sha256_email":"SHA256",
                             "SHA256EMAIL":"SHA256",
                             "SHA256Email":"email",
                             "emailaddress":"email",
                             "email_id":"email",
                             "SHA1EMAIL":"SHA1",
                             "hashed email":"hashed_email",
                             "SHA256HASHEDEMAIL":"SHA256",
                             "email2":"email",
                             "EMAIL_ADDRESS_1":"email",
                             "md5email":"MD5",
                             "hashedemail":"hashed_email",
                             "EMAIL_HASHED":"hashed_email",
                             "Broker Email":"email",
                             "HASH_MD5_LOWER_EMAIL":"MD5",
                             "profileEmail":"email",
                             "email_Address":"email",
                             "Hashed_Email":"hashed_email",
                             "PER_EMAIL":"email",
                             "Hashed_Email_Address":"email",
                             "Peloton Order Order Email":"email",
                             "EMAIL,FN,LN,,ZIP,CT,PHONE,":"email",
                             "Hashed_Email_ID_Hex":"hashed_email",
                             "#email_lc_md5":"MD5",
                             "MD5_EMAIL":"MD5",
                             "\EMAIL\"":"email",
                             "Customer Email":"email",
                             "CUSTOMERS_SalesOrder_BILLTOLookup::Email":"email",
                             "profile.email":"email",
                             "HASHED_EMAIL":"hashed_email",
                             "sha256email1":"SHA256",
                             "Hashed Email":"hashed_email",
                             "CUST_EMAIL":"email",
                             "SHA256 Email":"SHA256",
                             "email,fn,ln,st,zip,ct,phone,":"email",
                             "best_email":"email",
                             "LRI_G_EMAIL1":"email",
                             "SHA2_EMAIL":"SHA2",
                             "Md5 Email":"MD5",
                             "md5(email1)":"MD5",
                             "master_email":"email",
                             "HASHED_CUST_EMAIL":"hashed_email",
                             "REAL_EMAIL":"email",
                             "email_sha1":"SHA1",
                             "MD5 EMAIL":"MD5",
                             "Email/Email":"email",
                             "hash_email":"hashed_email",
                             "email md5":"MD5",
                             "Email_address":"email",
                             "Email1_SHA-1":"SHA1",
                             "MD5 Hashed Email":"MD5",
                             "EE_EMAIL_ADDRESS":"email",
                             "Email1 Address":"email",
                             "DMD_EMAIL":"email",
                             "QBOVRT_master_email":"email",
                             "lower(b.email_addr_hash_txt)":"hashed_email",
                             "Email1_SHA256":"SHA256",
                             "HashedEmail1":"hashed_email",
                             "CONTACT_EMAIL_ADDR":"email",
                             "Email ":"email",
                             "EmailAddressmd5":"MD5",
                             "CONTACT EMAIL":"email",
                             "MD5EMAIL1":"MD5",
                             "MD5email":"MD5",
                             "sha256email\",470121,624331,470121,UPHE - Onboarding - Salesforce Data\"":"SHA256",
                             "sha256email\",287259,579949,28236,Swirl McGarryBowen - Onboarding\"":"SHA256",
                             "AX_CUREMAIL_10":"email",
                             "Email ":"email",
                             "SHA256_email":"SHA256",
                             "USA_EMAIL":"email",
                             "Right_email":"email",
                             "MDG EMAIL":"MD5",
                             "email3":"email",
                             "sha256email\",287259,579949,28236,Swirl McGarryBowen - Onboarding\"":"SHA256",
                             "sha256email\",470121,624331,470121,UPHE - Onboarding - Salesforce Data\"":"SHA256",
                             "email_SHA_256":"SHA256",
                             "hashed_emailId":"hashed_email",
                             "EMAIL_ADDRESS1":"email",
                             "email_adr1":"email",
                             "EmailAddressHash_MD5":"MD5",
                             "LRI_G_email_address":"email",
                             "DANDRVINSON@EMAIL.COM":"email",
                             "user_email":"email",
                             "Email_hashed":"hashed_email",
                             "P_EMAIL":"email",
                             "Hashed email":"hashed_email",
                             "SHA250Email":"SHA256",
                             "sha256email\",287259,579949,28236,Swirl McGarryBowen - Onboarding\"":"SHA256",
                             "sha256email\",470121,624331,470121,UPHE - Onboarding - Salesforce Data\"":"SHA256",
                             "Email5":"email",
                             "Orders Customer Email":"email",
                             "Encrypted Email address":"hashed_email",
                             "eMail":"email",
                             "patron_email_addr":"email",
                             "Client_Email":"email",
                             "email_01":"email",
                             "emailhash_md5":"MD5",
                             "Person Account: Email":"email",
                             "Email_Appended":"email",
                             "Email_address_hash":"hashed_email",
                             "sha256email\",470121,624331,470121,UPHE - Onboarding - Salesforce Data\"":"SHA256",
                             "EMAIL_HASH":"hashed_email",
                             "Email ID":"email",
                             "email_billing":"email",
                             "PRIMARY_EMAIL_ADDRESS":"email",
                             "Opportunity Email":"email",
                             "CnPh_1_01_Email":"email",
                             "MD5_email":"MD5",
                             "email_address_256hash":"SHA256",
                             "EMAIL ADDR":"email",
                             "sha256email\",154996,542386,15314,StubHub - Salesforce Onboarding\"":"SHA256",
                             "Hash Email":"hashed_email",
                             "prmry_email_addr":"email",
                             "sha256email\",248499,508256,24412,Caesars - Salesforce RFP\"":"SHA256",
                             "MEMBER_EMAIL":"email",
                             "Sha1_Email":"SHA1",
                             "HASH EMAIL ADDR":"hashed_email",
                             "Sha1_Email":"SHA1",
                             "HASH EMAIL ADDR":"hashed_email",
                             "CUS_EMAIL_ADDRESS":"email",
                             "channels.email.address":"email",
                             "\Email1\"\"":"email",
                             "primaryEmail":"email",
                             "sha256email\",313789,533849,313789,Audi - Salesforce Imports\"":"SHA256",
                             "HASEHD_EMAIL":"hashed_email",
                             "\Email1\"\"":"email",
                             "hash email":"hashed_email",
                             "REGI_EMAIL_ADDR":"email",
                             "Customers Email":"email",
                             "email_addres":"email",
                             "sha256email\",116909,533639,11536,Weight Watchers - Onboarding\"":"SHA256",
                             "EMAIL-sha1":"SHA1",
                             "Email Address 1":"email",
                             "Contact Business Email":"email",
                             "\Email1\"\"":"email",
                             "Personal_email":"email",
                             "sha256email\",154996,542386,15314,StubHub - Salesforce Onboarding\"":"SHA256",
                             "EMAIL_MATCH_ID_1":"email",
                             "MD5 Email":"MD5",
                             "associatedemailaddress":"email",
                             "Email_Hash":"hashed_email",
                             "Email_Hash ":"hashed_email",
                             "EMAIL ADDRESS	":"email",
                             "EMAILADDR":"email",
                             "EMAILADDR ":"email",
                             "Contact: Primary Email":"email",
                             "Email Address - Prospects":"email",
                             "sha_email	":"hashed_email",
                             "sha_email":"hashed_email",
                             "primary_contact_primary_email":"email",
                             "EMAIL_ADDR_1":"email",
                             "Primary_Email_Address":"email",
                             "CUST_EMAIL_ADDR":"email",
                             "AlternateEmail":"email",
                             "PERSONEMAIL":"email",
                             "email_address_01":"email"
                            }, inplace = True)                       

### Follow up question: check cases where online audiences have email audience key label

## 4. CID-only audiences 

## 4.1 Results
### all audiences
enabled, non-FBR audiences where AK = CID and customer = enabled

audiences w/AK as CID - 5,746
-------------


### non-DS audiences
enabled, non-FBR audiences where AK = CID and customer = enabled

non DS audiences w/AK set as CID - 5,347
-------------


### DS audiences
enabled, non-FBR audiences where AK = CID and customer = enabled

DS audiences w/AK set as CID - 399
-------------


### 3.2.1 query all audiences with AK like '%id%' -> enabled, non-FBR audiences, enabled customers, not like: aaid, IDFA, cookie, device id, email, pel, identity_link, idl, liveramp, LR ID 1, LRID, MAID, mobile, order_id, PELID

#### CID definition methodology: after combing through AK labels strings that match AK like '%id%', IDs that do not resemble CIDs were removed. This analysis thus treats a CID as a synthetic identifier distinguished from IDLs/PELs, cookies, and device IDs.

```mysql
SELECT
  *
FROM
  audience_key_labels
INNER JOIN
  liveramp_customer_accounts
ON
  audience_key_labels.liveramp_customer_account_id = liveramp_customer_accounts.id
INNER JOIN
  customers
ON
  liveramp_customer_accounts.customer_id = customers.id
WHERE
  audience_key_labels.key_index = 0
  AND audience_key_labels.label LIKE '%ID%' 
  AND audience_key_labels.label NOT LIKE '%aaid%'
  AND audience_key_labels.label NOT LIKE '%idfa%'
  AND audience_key_labels.label NOT LIKE '%cookie%'
  AND audience_key_labels.label NOT LIKE '%device id%'
  AND audience_key_labels.label NOT LIKE '%Device_ID%'
  AND audience_key_labels.label NOT LIKE '%device%'
  AND audience_key_labels.label NOT LIKE '%email%'
  AND audience_key_labels.label NOT LIKE '%pel%'
  AND audience_key_labels.label NOT LIKE '%identity_link%'
  AND audience_key_labels.label NOT LIKE '%idl%'
  AND audience_key_labels.label NOT LIKE '%liveramp%'
  AND audience_key_labels.label NOT LIKE '%LR ID 1%'
  AND audience_key_labels.label NOT LIKE '%LRID%'
  AND audience_key_labels.label NOT LIKE '%MAID%'
  AND audience_key_labels.label NOT LIKE '%mobile%'
  AND audience_key_labels.label NOT LIKE '%order_id%'
  AND audience_key_labels.label NOT LIKE '%PELID%'
  AND audience_key_labels.label NOT LIKE '%FB_Ext_ID%'
  AND audience_key_labels.label NOT LIKE '%IdentityLink%'
  AND audience_key_labels.label NOT LIKE '%muid%'
  AND audience_key_labels.label NOT LIKE '%abilitec%'
  AND audience_key_labels.label NOT LIKE '%acxiom_id%'
  AND audience_key_labels.label NOT LIKE '%EXTERN_ID%'
  AND audience_key_labels.label NOT LIKE '%CNSMR_LINK_ID%'
  AND audience_key_labels.label NOT LIKE '%ttd_android%'
  AND customers.status = 1
  AND liveramp_customer_accounts.enabled = TRUE
  AND liveramp_customer_accounts.customer_link_audience = FALSE
GROUP BY
  liveramp_customer_accounts.id
```

In [44]:
CIDs_all_audiences = pd.read_csv('/Users/aschul/Documents/Documents - aschul1118mac/Code_Base/Investigations/20200417_AK_analysis/CID_AKs/CIDs.csv')

In [45]:
CIDs_all_audiences.head(2)

,id,liveramp_customer_account_id,key_index,label,id.1,customer_id,data_partner_thrift_identifier,display_name,volume_limit,updated_at,...,evaluation_tos_accepted_by,merged,query_rate_estimate,esp_key,reference_liveramp_customer_account_id,email_alias,custom_billing_rule_id,custom_billing_rule_type,encryption_key,enable_match_reports
0,11378,591,0,uid,591,199441,84,i360,NaN,2020-05-06 10:07:40,...,NaN,0,NaN,NaN,NaN,i360@liveramp.com,NaN,NaN,81aa0c2b5dd6c9563eb617d317df3378,0
1,868,1246,0,INDIVIDUAL_ID,1246,60794,142,Conde Nast,NaN,2020-03-17 18:41:38,...,NaN,0,NaN,NaN,61126.0,condenast@liveramp.com,NaN,NaN,72d194302037c9abace2ae0d35222259,0


In [46]:
CID_values = pd.DataFrame(CIDs_all_audiences["label"].value_counts())

In [47]:
CID_values.head(50)

,label
Customer ID,591
Client Customer ID,543
CustomerID,226
ID,223
Customer_ID,222
CCID,204
FICO INDIVIDUAL ID,163
CUSTOMERID,119
MBID,113
record_id,103


In [48]:
CID_values.info()

<class 'pandas.core.frame.DataFrame'>
Index: 809 entries, Customer ID to enc_pid
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   label   809 non-null    int64
dtypes: int64(1)
memory usage: 12.6+ KB


In [49]:
CID_values["label"]

Customer ID           591
Client Customer ID    543
CustomerID            226
ID                    223
Customer_ID           222
                     ... 
ODS_MEMBER_ID           1
wm_id                   1
Tidewater_ID            1
Client Contact ID       1
enc_pid                 1
Name: label, Length: 809, dtype: int64

### 3.2.1 query all audiences with AK like '%ID%' -> enabled, non-FBR audiences, enabled customers, data store

```mysql
SELECT
  liveramp_customer_accounts.id AS audience_id,
  liveramp_customer_accounts.display_name AS audience_name,
  customers.name AS customer_name,
  customers.id AS customer_id,
  liveramp_customer_accounts.type,
  audience_key_labels.label
FROM
  dms_segments
INNER JOIN
  lrc_field_definitions
ON
  dms_segments.lrc_field_definition_id = lrc_field_definitions.id
INNER JOIN
  liveramp_customer_accounts
ON
  lrc_field_definitions.lr_customer_account_id = liveramp_customer_accounts.id
INNER JOIN
  customers
ON
  liveramp_customer_accounts.customer_id = customers.id
INNER JOIN
  audience_key_labels
ON
  audience_key_labels.liveramp_customer_account_id = liveramp_customer_accounts.id
WHERE
  audience_key_labels.key_index = 0
  AND audience_key_labels.label LIKE '%ID%'
  AND audience_key_labels.label NOT LIKE '%aaid%'
  AND audience_key_labels.label NOT LIKE '%idfa%'
  AND audience_key_labels.label NOT LIKE '%cookie%'
  AND audience_key_labels.label NOT LIKE '%device id%'
  AND audience_key_labels.label NOT LIKE '%Device_ID%'
  AND audience_key_labels.label NOT LIKE '%device%'
  AND audience_key_labels.label NOT LIKE '%email%'
  AND audience_key_labels.label NOT LIKE '%pel%'
  AND audience_key_labels.label NOT LIKE '%identity_link%'
  AND audience_key_labels.label NOT LIKE '%idl%'
  AND audience_key_labels.label NOT LIKE '%liveramp%'
  AND audience_key_labels.label NOT LIKE '%LR ID 1%'
  AND audience_key_labels.label NOT LIKE '%LRID%'
  AND audience_key_labels.label NOT LIKE '%MAID%'
  AND audience_key_labels.label NOT LIKE '%mobile%'
  AND audience_key_labels.label NOT LIKE '%order_id%'
  AND audience_key_labels.label NOT LIKE '%PELID%'
  AND audience_key_labels.label NOT LIKE '%FB_Ext_ID%'
  AND audience_key_labels.label NOT LIKE '%IdentityLink%'
  AND audience_key_labels.label NOT LIKE '%muid%'
  AND audience_key_labels.label NOT LIKE '%abilitec%'
  AND audience_key_labels.label NOT LIKE '%acxiom_id%'
  AND audience_key_labels.label NOT LIKE '%EXTERN_ID%'
  AND audience_key_labels.label NOT LIKE '%CNSMR_LINK_ID%'
  AND audience_key_labels.label NOT LIKE '%ttd_android%'
  AND customers.status = 1
  AND liveramp_customer_accounts.enabled = TRUE
  AND liveramp_customer_accounts.customer_link_audience = FALSE
GROUP BY
  liveramp_customer_accounts.id
```

In [50]:
CIDs_DS_audiences = pd.read_csv('/Users/aschul/Documents/Documents - aschul1118mac/Code_Base/Investigations/20200417_AK_analysis/CID_AKs/CIDs_DS.csv')

In [51]:
CIDs_DS_audiences.head(5)

,audience_id,audience_name,customer_name,customer_id,type,label
0,345649,Custom Segments - MAIDS,180byTWO,531379,OnlineLiverampCustomerAccount,ad_id
1,470951,Arity - Data Store Segments,Arity,622131,OnlineLiverampCustomerAccount,adid
2,346269,Custom Segments - Appnexus ID's,180byTWO,531379,OnlineLiverampCustomerAccount,adnxs_id
3,253989,180byTwo - APP ID2 - AppNexus Cookie,180byTWO,531379,OnlineLiverampCustomerAccount,adnxs_id
4,447761,X-Mode - Data Store - CA,X-Mode,604579,OnlineLiverampCustomerAccount,advertiser_id


In [53]:
CID_values_DS_audiences = pd.DataFrame(CIDs_DS_audiences["label"].value_counts())

In [56]:
CID_values_DS_audiences.head(10)

,label
ID,50
rnc_reg_id,24
agile_id,20
id,19
CustomerID,18
AIQ_HHID,15
Client Customer ID,11
Customer ID,10
dedupe_id,9
stiristaid,9


### Follow up question: check which percentage of CID audiences are online vs offline

## 5. Address-only audiences